In [3]:
import torch as pt
from d2l import torch as d2l

pt.set_printoptions(2)  # 精简打印精度

## 生成多个锚框

In [4]:
# 待锚框的图像张量，最后两个维度是高和宽
# sizes：各种样式锚框的大小，即框的边占原图边的比例
# ratios：各种锚框的变形比例。为1时，锚框和原图长宽比例相同
def multibox_prior(imgs:pt.Tensor, sizes, ratios):
    in_h, in_w = imgs.shape[-2:]
    boxes_per_pixel = len(sizes) + len(ratios) - 1
    sizes = pt.tensor(sizes, device=imgs.device)
    ratios = pt.tensor(ratios, device=imgs.device)

    cen_h = (pt.arange(in_h, device=imgs.device)+.5) / in_h
    cen_w = (pt.arange(in_w, device=imgs.device)+.5) / in_w
    pixels_y, pixels_x = pt.meshgrid(cen_h, cen_w)
    pixels_y = pixels_y.reshape(-1)
    pixels_x = pixels_x.reshape(-1)

    w = pt.cat([(sizes * pt.sqrt(ratios[0])), (sizes[0] * pt.sqrt(ratios[1:]))]) * in_w
    h = pt.cat([(sizes / pt.sqrt(ratios[0])), (sizes[0] / pt.sqrt(ratios[1:]))]) * in_h

    boxes = pt.stack([-w, -h, w, h], dim=1).repeat(in_w*in_h, 1) / 2
    pixels = pt.stack([pixels_x, pixels_y, pixels_x, pixels_y], dim=1).repeat_interleave(boxes_per_pixel, dim=0)
    boxes += pixels
    return boxes.unsqueeze(0)

In [5]:
X = pt.rand(size=(2, 3, 561, 728))
Y = multibox_prior(X, sizes=[0.75, 0.5, 0.25], ratios=[1, 2, 0.5])
Y.shape

torch.Size([1, 2042040, 4])

In [17]:
boxes = Y.reshape(561, 728, -1, 4)

In [18]:
boxes = boxes[250, 250]

In [19]:
boxes[:,2]

tensor([273.34, 182.34,  91.34, 386.42, 193.38])

In [29]:
pt.max(boxes[:,:2], boxes[:, :2]).shape

torch.Size([5, 2])

In [26]:
pt.tensor([[1,2],[3,4]]).clamp(2, 3)

tensor([[2, 2],
        [3, 3]])